# Setting up the CNN model

For this project we are using Python, and Jupyter Notebooks hosted on Google Colab. Any Jupyter engine will can be used, but it will require adjustmets to this initial setup to be able to access the raw data and the helper tools. 

To keep the notebook clean, all the implementation details are hidden in a set of helper functions in the file `deeppredict_tools.py`

### Prerequisites for running this Notebook on Google Colab 

We need to mount a Google Drive folder called `deeppredict`, which has the same structure and contents as the repository at https://github.com/dimitardi/deep-predict. 

And the matlab raw data files in the subfolder `DEEPPREDICT_HOME/dataset` (see `deeppredict_0_download_raw_data.ipynb` for how to download these files)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

DEEPPREDICT_HOME = '/content/gdrive/My Drive/deeppredict'

# add the home folder to the python path to be able to import and use the included code
import sys
sys.path.append(DEEPPREDICT_HOME)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install --upgrade -q -r '/content/gdrive/My Drive/deeppredict/requirements.txt'

import skimage as sk
# explicitly check the sci-kit image version
# currently colab has the 0.13 by default, which is too old. It should be at least >= 0.14
sk.__version__

'0.14.2'

### CNN Architecture

For this use-case we chose an architecture mimicking the Guo et al. one, described in the research article [Deep Learning Enabled Fault Diagnosis Using Time-Frequency Image Analysis of Rolling Element Bearings](https://www.hindawi.com/journals/sv/2017/5067651/)

We use the following hyperparameters:

In [3]:
from keras import optimizers

## input data parameters
IMAGES_SHAPE = (96, 96, 4)
PADDING = 'same'
KERNEL_SIZE = (5, 5)
KERNEL_INITIALIZER = 'glorot_normal'
# parameters for deep layers
NUMBER_OF_CLASSES = 3  # N, IR and B
DROPOUT = 0.5
LEAK_ALPHA = 0.1
MAX_POOLING_POOL_SIZE = (2, 2)
ACTIVATION_LAYER_FUNCTION = 'softmax'
# loss and optimizer
LOSS_FUNCTION = 'categorical_crossentropy'
LEARNING_RATE = 0.001
OPTIMIZER = optimizers.Adam(LEARNING_RATE)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


The reasons behind the choice of the hyperparameters are too broad to describe here. 

However, an example of how we found a good learning rate using the **cyclical learning rate** approach is described [in this Jupyter Notebook from our repository](https://github.com/dimitardi/deep-predict/blob/master/deeppredict_A1_finding_learning_rate.ipynb).

In [4]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU
from keras import metrics


# CNN architecture from Guo et al.
model = Sequential()
model.add(Conv2D(5, KERNEL_SIZE,                  
                 input_shape=IMAGES_SHAPE,
                 data_format='channels_last',
                 kernel_initializer=KERNEL_INITIALIZER,                 
                 padding=PADDING))
model.add(LeakyReLU(LEAK_ALPHA))
model.add(Conv2D(10, KERNEL_SIZE, 
                 kernel_initializer=KERNEL_INITIALIZER,
                 padding=PADDING))
model.add(LeakyReLU(LEAK_ALPHA))
model.add(MaxPooling2D(pool_size=MAX_POOLING_POOL_SIZE))
model.add(Conv2D(10, KERNEL_SIZE, 
                 kernel_initializer=KERNEL_INITIALIZER,
                 padding=PADDING))
model.add(LeakyReLU(LEAK_ALPHA))
model.add(MaxPooling2D(pool_size=MAX_POOLING_POOL_SIZE))
model.add(Flatten())
model.add(Dense(10))
model.add(LeakyReLU(LEAK_ALPHA))
model.add(Dense(5))
model.add(LeakyReLU(LEAK_ALPHA))
model.add(Dropout(DROPOUT))
model.add(Dense(NUMBER_OF_CLASSES))
model.add(Activation(ACTIVATION_LAYER_FUNCTION))
  
model.compile(loss=LOSS_FUNCTION, 
              optimizer=OPTIMIZER,
              metrics=[metrics.categorical_accuracy])

filename_for_model = f'{DEEPPREDICT_HOME}/models/CNN_arch_guo_shape96-96-4-3__initial.h5'
model.save(filename_for_model)
print(f'Saved a new initial model at {filename_for_model}')
  
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Saved a new initial model at /content/gdrive/My Drive/deeppredict/models/CNN_arch_guo_shape96-96-4-3__initial.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 5)         505       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 96, 96, 5)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 10)        1260      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 96, 96, 10)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 10)        0         
______________________________________________